In [22]:
import numpy as np
indices = np.random.choice(10, 5, replace=True)
arr = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
print(arr[indices])
print(arr[-11:])
reward_buffer = [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, -1]
size = 7
game_start_index = 0
reward = reward_buffer[size - 1]
updated_rewards = np.empty((size - game_start_index,), int)
updated_rewards[::2] = 1 * reward
updated_rewards[1::2] = -1 * reward
updated_rewards = np.flip(updated_rewards)
print(updated_rewards)
reward_buffer[game_start_index:size] = updated_rewards
print(reward_buffer)
game_start_index = 7
size = 15
print(len(reward_buffer))
reward = reward_buffer[size - 1]
updated_rewards = np.empty((size - game_start_index,), int)
updated_rewards[::2] = 1 * reward
updated_rewards[1::2] = -1 * reward
updated_rewards = np.flip(updated_rewards)
print(updated_rewards)
reward_buffer[game_start_index:size] = updated_rewards
print(reward_buffer)
print(len(reward_buffer))
overflow_reward = 1
overflow_count = 1

[8 5 9 7 3]
[ 1  2  3  4  5  6  7  8  9 10]
[ 1 -1  1 -1  1 -1  1]
[1, -1, 1, -1, 1, -1, 1, 0, 0, 0, 0, 0, 0, 0, -1]
15
[ 1 -1  1 -1  1 -1  1 -1]
[1, -1, 1, -1, 1, -1, 1, 1, -1, 1, -1, 1, -1, 1, -1]
15


In [23]:
# reward_buffer = np.array([1, -1, 1, -1, 1, -1, 0])
# reward_buffer[6] = [1, 1, 1, 1]
# print(reward_buffer)

In [20]:
# from alphazero_agent import AlphaZeroAgent
import gymnasium as gym 
import tensorflow as tf
import numpy as np
# import gym_envs

class ClipReward(gym.RewardWrapper):
    def __init__(self, env, min_reward, max_reward):
        super().__init__(env)
        self.min_reward = min_reward
        self.max_reward = max_reward
        self.reward_range = (min_reward, max_reward)

    def reward(self, reward):
        return np.clip(reward, self.min_reward, self.max_reward)


# env = ClipReward(gym.wrappers.AtariPreprocessing(gym.make("MsPacmanNoFrameskip-v4", render_mode="rgb_array"), terminal_on_life_loss=True), -1, 1) # as recommended by the original paper, should already include max pooling
env = TicTacToeEnv()
# env = gym.make("MsPacmanNoFrameskip-v4", render_mode="rgb_array")
env = gym.wrappers.FrameStack(env, 4)

config = {
        'activation': 'relu',
        'kernel_initializer': 'orthogonal',
        'optimizer_function': tf.keras.optimizers.legacy.Adam,
        'learning_rate': 0.2,
        'adam_epsilon': 1e-7,
        'clipnorm': 0.5,
        # NORMALIZATION?
        # REWARD CLIPPING
        'num_epochs': 30,
        'num_filters': 32,
        'kernel_size': 3,
        'stride': 1,
        'num_res_blocks': 5,
        'critic_conv_filters': 32,
        'critic_conv_layers': 1,
        'critic_dense_size': 32,
        'critic_dense_layers': 1,
        'actor_conv_filters': 32,
        'actor_conv_layers': 1,
        'actor_dense_size': 32,
        'actor_dense_layers': 1,
        'memory_size': 180,
        'max_game_length': 9,
        'batch_size': 9,
        'dirichlet_alpha': 0.3,
        'dirichlet_epsilon': 0.25,
        'c_puct': 1,
        'monte_carlo_simulations': 18,
        'two_player': True,
        'weight_decay': 1e-4,
    }    



NameError: name 'TicTacToeEnv' is not defined

In [24]:
agent = AlphaZeroAgent(env, config=config)
agent.train()

TypeError: AlphaZeroAgent.__init__() missing 1 required positional argument: 'name'

In [25]:
array = np.zeros((3, 9))
array[0] = [1, 2, 3, 4, 5, 6, 7, 8, 9]
print(array)

[[1. 2. 3. 4. 5. 6. 7. 8. 9.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [26]:
import gym_envs
import gymnasium as gym
env = gym.make('gym_envs/TicTacToe-v0')

state, info = env.reset()
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
env.render()
state, reward, terminated, truncated, info = env.step(0)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(4)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(3)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(6)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(2)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(1)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(7)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(8)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(5)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
print("Truncated:", truncated)
env.render()


env.reset()
state, reward, terminated, truncated, info = env.step(0)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(3)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(7)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(4)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(2)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(6)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(1)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
print("Truncated:", truncated)
print("Reward:", reward)



[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
Turn:  0.0
Legal moves:  [0 1 2 3 4 5 6 7 8]
[[[0. 1. 1.]
  [0. 0. 1.]
  [0. 0. 1.]]

 [[0. 0. 1.]
  [0. 0. 1.]
  [0. 0. 1.]]

 [[0. 0. 1.]
  [0. 0. 1.]
  [0. 0. 1.]]]
Turn:  0.0
Legal moves:  [1 2 3 4 5 6 7 8]
Terminated: False
[[[1. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 1. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
Turn:  0.0
Legal moves:  [1 2 3 5 6 7 8]
Terminated: False
[[[0. 1. 1.]
  [0. 0. 1.]
  [0. 0. 1.]]

 [[0. 1. 1.]
  [1. 0. 1.]
  [0. 0. 1.]]

 [[0. 0. 1.]
  [0. 0. 1.]
  [0. 0. 1.]]]
Turn:  0.0
Legal moves:  [1 2 5 6 7 8]
Terminated: False
[[[1. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[1. 0. 0.]
  [0. 1. 0.]
  [0. 0. 0.]]

 [[0. 1. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
Turn:  0.0
Legal moves:  [1 2 5 7 8]
Terminated: False
[[[0. 1. 1.]
  [0. 0. 1.]
  [0. 1. 1.]]

 [[0. 1. 1.]
  [1. 0. 1.]
  [0. 0. 1.]]

 [[1. 0. 1.]
  [0. 0. 1.]


In [27]:
# import tensorflow as tf
# regularizer = tf.keras.regularizers.L2(2)
# print(regularizer(agent.model.trainable_variables))

In [28]:
import numpy as np
import tensorflow as tf
arr = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

print(arr/np.sum(arr))
# print(tf.nn.softmax(arr))

[0.02222222 0.04444444 0.06666667 0.08888889 0.11111111 0.13333333
 0.15555556 0.17777778 0.2       ]


In [1]:
from alphazero_agent import AlphaZeroAgent
# from alphazero_agent import AlphaZeroAgent
import gymnasium as gym
import tensorflow as tf
import numpy as np
import gym_envs

class ClipReward(gym.RewardWrapper):
    def __init__(self, env, min_reward, max_reward):
        super().__init__(env)
        self.min_reward = min_reward
        self.max_reward = max_reward
        self.reward_range = (min_reward, max_reward)

    def reward(self, reward):
        return np.clip(reward, self.min_reward, self.max_reward)


# env = ClipReward(gym.wrappers.AtariPreprocessing(gym.make("MsPacmanNoFrameskip-v4", render_mode="rgb_array"), terminal_on_life_loss=True), -1, 1) # as recommended by the original paper, should already include max pooling
# env = TicTacToeEnv(render_mode="rgb_array")
# env = gym.make("MsPacmanNoFrameskip-v4", render_mode="rgb_array")
# env = gym.wrappers.FrameStack(env, 4)
env = gym.make('gym_envs/Connect4-v0', render_mode="rgb_array")


# MODEL SEEMS TO BE UNDERFITTING SO TRY AND GET IT TO OVERFIT THEN FIND A HAPPY MEDIUM
# 1. INCREASE THE NUMBER OF RESIDUAL BLOCKS
# 2. INCREASE THE NUMBER OF FILTERS
# 3. DECREASE REGULARIZATION
# 4. TRY DECREASING LEARNING RATE (maybe its that whole thing where the policy goes to like 1 0 0 0 0... etc and then goes back on the third training step, so maybe the learning rate is too high)
# 5. TO OVERFIT USE LESS DATA (but that is probably just a bad idea)
# config = {
#         'activation': 'relu',
#         'kernel_initializer': 'glorot_uniform',
#         'optimizer': tf.keras.optimizers.legacy.Adam,
#         'learning_rate': 0.001, # 0.00001 could maybe increase by a factor of 10 or 100 and try to do some weights regularization
#         'adam_epsilon': 3.25e-6,
#         'clipnorm': None,
#         # NORMALIZATION?
#         # REWARD CLIPPING
#         'training_steps': 40,
#         'num_filters': 256,
#         'kernel_size': 3,
#         'stride': 1,
#         'num_res_blocks': 20,
#         'critic_conv_filters': 32, # 1
#         'critic_conv_layers': 1,
#         'critic_dense_size': 256,
#         'critic_dense_layers': 1,
#         'actor_conv_filters': 32, # 
#         'actor_conv_layers': 1,
#         'actor_dense_size': 0,
#         'actor_dense_layers': 0,
#         'replay_buffer_size': 800, # IN GAMES
#         'replay_batch_size': 50, # IN MOVES
#         'root_dirichlet_alpha': 0.5, # 2 in theory?
#         'root_exploration_fraction': 0, # 0.25 in paper
#         'pb_c_base': 500,
#         'pb_c_init': 2,
#         'num_simulations': 200,
#         # 'two_player': True,
#         'weight_decay': 0.00, # could try setting this to something other than 0 and increasing learning rate
#         'num_sampling_moves': 0, 
#         'initial_temperature': 1,
#         'exploitation_temperature': 0.1,
#         'value_loss_factor': 1, # could try setting this to something other than 1
#         'games_per_generation': 10, # times 8 from augmentation
#     }

config = {
        'activation': 'relu',
        'kernel_initializer': 'glorot_uniform',
        'optimizer': tf.keras.optimizers.legacy.Adam,
        'min_learning_rate': 0.0005, #0.0001 # 0.00001 could maybe increase by a factor of 10 or 100 and try to do some weights regularization
        'max_learning_rate': 0.0005, #0.0001
        'number_of_lr_cycles': 1, # this will determine the step size based on training steps
        # STILL ADD A SCHEDULE FOR BASE LEARNING RATE (MIN LEARNING RATE)
        'adam_epsilon': 3.25e-6,
        'clipnorm': None,
        # NORMALIZATION?
        # REWARD CLIPPING
        'training_steps': 100, # alpha zero did 700,000, the lessons from alpha zero did 40 generations but 1000 batches per generation, so 40,000 batches (they just had a cyclical learning rate per generation (also they trained twice on the same data every generation))
        'num_filters': 256,
        'kernel_size': 3,
        'stride': 1,
        'num_res_blocks': 20,
        'critic_conv_filters': 32, # 1
        'critic_conv_layers': 1,
        'critic_dense_size': 256,
        'critic_dense_layers': 1,
        'actor_conv_filters': 32, # 
        'actor_conv_layers': 1,
        'actor_dense_size': 0,
        'actor_dense_layers': 0,
        'replay_buffer_size': 100, # IN GAMES
        'replay_batch_size': 240, # SHOULD BE ROUGHLY SAME AS AVERAGE MOVE PER GENERATION (SO LIKE 7 TIMES NUMBER OF GAMES PLAYED PER GENERATION) <- what was used in the original paper (they played 44M games, 50 moves per game and sampled 700,000 minibatches of size 4096 (so thats like sampling 1 time per move roughly but this was also happening with parrallel data collection i believe))
        'games_per_generation': 10, # times 8 from augmentation
        'root_dirichlet_alpha': 2.5, # Less than 1 more random, greater than one more flat # 2 in theory? # 0.3 in alphazero for chess # TRY CHANGING (MAYBE LOWER? (IT SEEMS TO PLAY THE SAME LINE OVER AND OVER AGAIN <- so we want a lesss flat distribution maybe)
        'root_exploration_fraction': 0.25, # 0.25 in paper
        'pb_c_base': 20000, # Seems unimportant to be honest (increases puct the more simulations there are)
        'pb_c_init': 1.25, # 1.25 in paper # MAYBE HIGHER? (IT SEEMS TO PLAY THE SAME LINE OVER AND OVER AGAIN)
        'num_simulations': 800, # INCREASE THIS since the model is missing 1 move wins (and also 2 and 3 move wins (it wins by luck)))
        # 'two_player': True,
        'weight_decay': 0.00001, # could try setting this to something other than 0 and increasing learning rate
        'num_sampling_moves': 30, 
        'initial_temperature': 1,
        'exploitation_temperature': 0.1,
        'value_loss_factor': 1, # could try setting this to something other than 1
    }

agent = AlphaZeroAgent(env, config=config, name="alphazero")

Box(0.0, 1.0, (6, 7, 3), float64)


C:\Users\adria\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [2]:
agent.model.load_weights("./alphazero.keras")
agent.train()

Training Step  1
training: 0
Game  1
Predicted Policy  [0.14285715 0.14285715 0.14285715 0.14285715 0.14285715 0.14285715
 0.14285715]
Predicted Value  0.0
root sent: <alphazero.alphazero_mcts.Node object at 0x000001E64E69D590>
Action  2
root_received : <alphazero.alphazero_mcts.Node object at 0x000001E64E69D590> {0: <alphazero.alphazero_mcts.Node object at 0x000001E65038FAD0>, 1: <alphazero.alphazero_mcts.Node object at 0x000001E64F8DFC50>, 2: <alphazero.alphazero_mcts.Node object at 0x000001E65084A490>, 3: <alphazero.alphazero_mcts.Node object at 0x000001E65084BC10>, 4: <alphazero.alphazero_mcts.Node object at 0x000001E650293910>, 5: <alphazero.alphazero_mcts.Node object at 0x000001E650297290>, 6: <alphazero.alphazero_mcts.Node object at 0x000001E65088F050>}
child root selected: <alphazero.alphazero_mcts.Node object at 0x000001E65084A490>
Target Policy [0.04625    0.05875    0.66874999 0.05875    0.05625    0.05375
 0.0575    ]
Predicted Policy  [0.09860864 0.09034191 0.3584221  0.00